<a href="https://colab.research.google.com/github/sbh1893/Gukboseong_Geumsok/blob/main/ex01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import streamlit as st
import pandas as pd
import io
from openpyxl import load_workbook
from openpyxl.styles import Alignment, Border, Side, PatternFill, Font

# 페이지 설정
st.set_page_config(page_title="엑셀 자동 정리기", layout="wide")

st.title("📊 사급 목록 엑셀 자동 정리기")
st.write("엑셀 또는 CSV 파일을 업로드하면, 업체별/일자별로 정리하고 서식을 적용해줍니다.")

# 1. 파일 업로드 위젯
uploaded_file = st.file_uploader("여기에 파일을 드래그하거나 클릭해서 업로드하세요.", type=['xlsx', 'csv'])

if uploaded_file is not None:
    st.success("파일 업로드 성공! 변환을 시작합니다...")

    # 2. 데이터 읽기 (만능 로더 함수)
    @st.cache_data # 속도 향상을 위해 캐싱
    def load_data(file):
        df = None
        # 파일 포인터를 처음으로 되돌림 (중요)
        file.seek(0)

        # 1차 시도: 엑셀로 읽기
        try:
            df = pd.read_excel(file, header=2)
            return df, "Excel"
        except:
            pass

        # 2차 시도: CSV로 읽기
        encodings = ['utf-8', 'cp949', 'euc-kr']
        for enc in encodings:
            try:
                file.seek(0)
                df = pd.read_csv(file, header=2, encoding=enc)
                return df, f"CSV({enc})"
            except:
                pass
        return None, "Fail"

    df_raw, msg = load_data(uploaded_file)

    if df_raw is not None:
        # 3. 데이터 가공 로직
        try:
            # 컬럼명 정리
            df_raw.columns = df_raw.columns.astype(str).str.strip()

            # 불필요한 행 제거
            if '규 격' in df_raw.columns:
                df = df_raw[~df_raw['규 격'].astype(str).str.contains('합계', na=False)].copy()
            else:
                df = df_raw.copy()

            # 컬럼 찾기
            company_col = '발 주 처' if '발 주 처' in df.columns else '발주처'
            date_col = '납품일'

            if company_col in df.columns and date_col in df.columns:
                # 빈 값 채우기 & 날짜 변환
                df[date_col] = df[date_col].ffill()
                df[company_col] = df[company_col].ffill()
                df[date_col] = pd.to_datetime(df[date_col], errors='coerce').dt.strftime('%Y-%m-%d')

                # 재정렬
                df_sorted = df.sort_values(by=[company_col, date_col])
                cols = [company_col, date_col] + [c for c in df_sorted.columns if c not in [company_col, date_col]]
                df_final = df_sorted[cols].set_index([company_col, date_col])

                # 미리보기 보여주기
                st.write("### 👇 결과 미리보기")
                st.dataframe(df_final)

                # 4. 엑셀 스타일 적용 및 다운로드 준비
                # 메모리 내에서 엑셀 파일 생성 (io.BytesIO)
                output = io.BytesIO()

                # Pandas -> ExcelWriter (Openpyxl 엔진)
                with pd.ExcelWriter(output, engine='openpyxl') as writer:
                    df_final.to_excel(writer, sheet_name='Sheet1')

                # 스타일 적용을 위해 다시 로드 (메모리 상에서)
                output.seek(0)
                wb = load_workbook(output)
                ws = wb.active

                # 스타일 정의
                thin_border = Border(left=Side(style='thin'), right=Side(style='thin'),
                                     top=Side(style='thin'), bottom=Side(style='thin'))
                header_fill = PatternFill(start_color="D3D3D3", end_color="D3D3D3", fill_type="solid")
                center_align = Alignment(horizontal='center', vertical='center')

                # 스타일 적용 루프
                for row in ws.iter_rows():
                    for cell in row:
                        cell.border = thin_border
                        cell.alignment = center_align
                        if cell.row == 1: # 헤더
                            cell.fill = header_fill
                            cell.font = Font(bold=True)

                # 병합된 셀 정렬 수정
                for merge in list(ws.merged_cells):
                    ws[merge.coord.split(':')[0]].alignment = center_align

                # 컬럼 너비 조정
                for col in ws.columns:
                    max_length = 0
                    column = col[0].column_letter
                    for cell in col:
                        try:
                            if len(str(cell.value)) > max_length:
                                max_length = len(str(cell.value))
                        except:
                            pass
                    ws.column_dimensions[column].width = (max_length + 2) * 1.2

                # 최종 결과물 저장
                output = io.BytesIO()
                wb.save(output)
                output.seek(0)

                # 5. 다운로드 버튼 생성
                st.download_button(
                    label="📥 엑셀 파일 다운로드",
                    data=output,
                    file_name="정리된_사급목록.xlsx",
                    mime="application/vnd.openxmlformats-officedocument.spreadsheetml.sheet"
                )

            else:
                st.error(f"필수 컬럼을 찾을 수 없습니다. (감지된 컬럼: {df.columns.tolist()})")

        except Exception as e:
            st.error(f"데이터 처리 중 오류가 발생했습니다: {e}")

    else:
        st.error("파일을 읽을 수 없습니다. 형식을 확인해주세요.")